### Load libs

In [1]:
import pandas as pd
import numpy as np
import sklearn
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
import re
from scipy import spatial
import time
import multiprocessing

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhiti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load data

In [2]:
df1=pd.read_excel(r"datasets/extremism.xlsx")

df=pd.DataFrame()
df['id'] = df1['id']
df['text'] = df1['text']
df['class'] = df1['class']
df = df.dropna()

In [3]:
df

,id,text,class
0,1,Таким образом никто не внес большего вклад...,3.0
1,2,"Можно с уверенностью сказать, что все полезное...",3.0
2,3,Славяне должны на нас работать. Если они нам б...,3.0
3,4,"Русское государство росло, развивалось из свои...",3.0
4,5,Поставим власть под контроль народа,3.0
...,...,...,...
304,305,Социальное. Соответствующий подход предполагае...,0.0
305,306,Рост продуктивности человеческой деятельности ...,0.0
306,307,"Социальная дифференциация, в том числе по этни...",0.0
307,308,Демографический рост и расширение экономическо...,0.0


### Load models

In [19]:
w2v_model = Word2Vec(
    min_count=10,
    window=10,
    size=300,
    negative=10,
    alpha=0.5,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1,workers=4)

exp_model = w2v_model.wv.load_word2vec_format(r'models/exp_model.bin', binary=True)

In [52]:
exp_model.most_similar('кошка')

[('соседка', 0.8491843938827515),
 ('собачка', 0.8448320031166077),
 ('леся', 0.841035008430481),
 ('насквозь', 0.817664623260498),
 ('болеет', 0.8155531883239746),
 ('пьяная', 0.8155224323272705),
 ('бедная', 0.8139984011650085),
 ('тётя', 0.8106111288070679),
 ('мелкая', 0.8079123497009277),
 ('достала', 0.8047630786895752)]

## Prepare funcs

### Tokenization

In [21]:
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

def comma2space(word):
    new_word = ''
    try:
        for char in word:
            if char in patterns:
                char = ' '
            new_word += char
    except TypeError:
        pass
    return new_word
    
def lemmatize(doc):
    doc = re.sub(patterns, '', doc)
    tokens = []
    for token in doc.split():
        if token:
            
            token = token.lower()
            token = token.strip()
            tokens.append(token)
    return tokens

def dropstopwords(docs):
    stopwords = nltk.corpus.stopwords.words('russian')
    sen =  []
    for doc in docs:
        if (doc not in stopwords):
            sen.append(doc)
    return sen

def dropcomma(doc):
    new_sen = ''
    for char in doc:
        if (char not in patterns):
            new_sen += char
    return new_sen

def tokenize(docs):
    docs = comma2space(docs)
    docs = lemmatize(docs)
    docs = dropstopwords(docs)
    return docs

### Medium word-vector

In [22]:
def mediumVector(sen):
    medVec = exp_model["хачи"]
    unexceptedVectors = 0
    for word in sen:
        vec = False
        try:
            vec = exp_model.word_vec(word)
            medVec = medVec + vec
        except:
            unexceptedVectors += 1
            
    if unexceptedVectors != 0 :
        print('Lost - ' + str(unexceptedVectors))
    elif unexceptedVectors == len(sen)-1:
        print('Sentence lost')
        
    medVec = medVec - exp_model["хачи"]
    medVec = medVec / (len(sen) - unexceptedVectors)
    return medVec

## Organise data 

In [23]:
data=df['text'].apply(tokenize)
data

0      [таким, образом, никто, внес, большего, вклада...
1      [уверенностью, сказать, полезное, хорошее, сов...
2      [славяне, должны, работать, нам, нужны, могут,...
3      [русское, государство, росло, развивалось, сво...
4                   [поставим, власть, контроль, народа]
                             ...                        
304    [социальное, соответствующий, подход, предпола...
305    [рост, продуктивности, человеческой, деятельно...
306    [социальная, дифференциация, числе, этническом...
307    [демографический, рост, расширение, экономичес...
308    [соответственно, политика, возникает, связи, у...
Name: text, Length: 309, dtype: object

In [24]:
df['tokenized']=df['text'].apply(tokenize)
df['medVec']=df['tokenized'].apply(mediumVector)
df

Lost - 6
Lost - 4
Lost - 11
Lost - 10
Lost - 1
Lost - 1
Lost - 1
Lost - 2
Lost - 42
Lost - 6
Lost - 11
Lost - 6
Lost - 2
Lost - 7
Lost - 6
Lost - 5
Lost - 7
Lost - 118
Lost - 5
Lost - 5
Lost - 7
Lost - 2
Lost - 4
Lost - 14
Lost - 5
Lost - 10
Lost - 6
Lost - 2
Lost - 6
Lost - 5
Lost - 6
Lost - 4
Lost - 1
Lost - 1
Lost - 3
Lost - 7
Lost - 16
Lost - 6
Lost - 17
Lost - 3
Lost - 1
Lost - 6
Lost - 3
Lost - 4
Lost - 1
Lost - 9
Lost - 7
Lost - 4
Lost - 4
Lost - 9
Lost - 6
Lost - 4
Lost - 3
Lost - 5
Lost - 3
Lost - 8
Lost - 4
Lost - 3
Lost - 1
Lost - 6
Lost - 1
Lost - 1
Lost - 1
Lost - 2
Lost - 2
Lost - 3
Lost - 12
Lost - 4
Lost - 2
Lost - 6
Lost - 4
Lost - 3
Lost - 3
Lost - 8
Lost - 3
Lost - 2
Lost - 5
Lost - 2
Lost - 1
Lost - 4
Lost - 2
Lost - 1
Lost - 2
Lost - 3
Lost - 5
Lost - 1
Lost - 1
Lost - 3
Lost - 7
Lost - 9
Lost - 14
Lost - 5
Lost - 4
Lost - 6
Lost - 5
Lost - 7
Lost - 5
Lost - 4
Lost - 10
Lost - 5
Lost - 1
Lost - 1
Lost - 50
Lost - 6
Lost - 11
Lost - 11
Lost - 7
Lost - 1
Lost - 5
Los

,id,text,class,tokenized,medVec
0,1,Таким образом никто не внес большего вклад...,3.0,"[таким, образом, никто, внес, большего, вклада...","[-0.040627204, -0.015365396, -0.047234748, -0...."
1,2,"Можно с уверенностью сказать, что все полезное...",3.0,"[уверенностью, сказать, полезное, хорошее, сов...","[-0.039338265, -0.028398223, -0.0673505, -0.01..."
2,3,Славяне должны на нас работать. Если они нам б...,3.0,"[славяне, должны, работать, нам, нужны, могут,...","[-0.036592077, -0.06395722, -0.045187872, -0.0..."
3,4,"Русское государство росло, развивалось из свои...",3.0,"[русское, государство, росло, развивалось, сво...","[-0.01085669, -0.022112658, -0.07141073, -0.03..."
4,5,Поставим власть под контроль народа,3.0,"[поставим, власть, контроль, народа]","[-0.052434314, -0.07669518, -0.019612849, -0.0..."
...,...,...,...,...,...
304,305,Социальное. Соответствующий подход предполагае...,0.0,"[социальное, соответствующий, подход, предпола...","[-0.03102859, -0.050743006, -0.037808374, -0.0..."
305,306,Рост продуктивности человеческой деятельности ...,0.0,"[рост, продуктивности, человеческой, деятельно...","[-0.022473654, -0.052780617, -0.06827721, -0.0..."
306,307,"Социальная дифференциация, в том числе по этни...",0.0,"[социальная, дифференциация, числе, этническом...","[-0.017352613, -0.019620007, -0.04677039, -0.0..."
307,308,Демографический рост и расширение экономическо...,0.0,"[демографический, рост, расширение, экономичес...","[-0.03736097, -0.06539034, -0.062378306, -0.05..."


In [13]:
np.shape(df['medVec'][0])

(300,)

In [14]:

w2v_model = Word2Vec(
    min_count=10,
    window=10,
    size=300,
    negative=10,
    alpha=0.5,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1,workers=4)

#BUILD_VOCAB()
t = time.time()
w2v_model.build_vocab(data, progress_per=100)
print("Time to build vocab': {} mins".format(round((time.time() - t) / 60, 2)))

#TRAIN()
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))

Time to build vocab': 0.0 mins
Time to train the model: 0.01 mins


In [48]:
w2v_model.init_sims(replace=True)
w2v_model.wv.save_word2vec_format('hachi.bin', binary=True)

In [29]:
model1 = w2v_model.wv.load_word2vec_format(r'models\hachi.bin', binary=True)
model1

In [44]:
model1.most_similar(positive=["хуй","русских"])

KeyError: "word 'хуй' not in vocabulary"

In [65]:
import numpy as np

index2word_set = set(model1.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

<ipython-input-65-37b38b84b612>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(model1.wv.index2word)


In [ ]:

from pathlib import Path
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras import optimizers


# build a model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=10000))
model.add(Dropout(0.25))
model.add(Dense(50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
optimiz = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss = 'binary_crossentropy',optimizer = optimiz ,metrics = ['accuracy'])

# convert texts to TF-IDF matrices


# fit the model
#hist = model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 5,batch_size = 64)

In [ ]:
listok=['хач','убивай ',"негр"]
deist='всех'
score=[]
slovar={}
for i in range(len(listok)):
    s1_afv = avg_feature_vector(deist, model=model1, num_features=300, index2word_set=index2word_set)
    s2_afv = avg_feature_vector(listok[i], model=model1, num_features=300, index2word_set=index2word_set)
    sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
    score.append(sim)
    slovar.update({deist+' / '+listok[i]:sim})
sorted_tuples = sorted(slovar.items(), key=lambda item: item[1])

sorted_dict = {k: v for k, v in sorted_tuples}

print(sorted_dict)  # {1: 1, 3: 4, 2: 9}